In [82]:
!which python

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/bin/python


In [81]:
#!source activate py34
!source activate py36

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
/usr/bin/sh: line 0: source: activate: file not found


In [83]:
import sys

#sys.path.append('/home/faske/g/anaconda3/envs/py34/lib/python3.4/site-packages')
sys.path.append('/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages')
sys.path.append("/data/gpfs/assoc/parchmanlab/tfaske/ipynb/include_utils")

import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
#import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE, call, check_output
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
#import pysam
from collections import OrderedDict, namedtuple, Counter
import operator
import multiprocessing as mp
import shutil
import tempfile
#from ipyparallel import Client
import scandir
import glob
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import pickle
#import Levenshtein as lv

In [84]:
vcfutils = "vcfutils.pl"
vcftools = "vcftools"
bcftools = "bcftools"
bgzip = "bgzip"
tabix = "tabix"
java = "java"
plink = "plink --noweb"

In [85]:
analysis_dir = '/data/gpfs/home/tfaske/g/balsam/ddocent/filtering'

In [86]:
cd $analysis_dir

/data/gpfs/assoc/parchmanlab/tfaske/balsam/ddocent/filtering


In [87]:
vcf_file = os.path.join(analysis_dir, "TotalRawSNPs.vcf.gz")
assert os.path.exists(vcf_file)
vcf_file

'/data/gpfs/home/tfaske/g/balsam/ddocent/filtering/TotalRawSNPs.vcf.gz'

# Filter first round 
    missing data, MAF, biallelic

In [88]:
!$vcftools --remove-indels \
--max-missing 0.5 \
--min-alleles 2 \
--max-alleles 2 \
--maf 0.01 \
--thin 5 \
--remove-filtered-all \
--recode \
--recode-INFO-all \
--gzvcf \
$vcf_file \
--out $'balsam_miss50_thin5_MAF1'


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/TotalRawSNPs.vcf.gz
	--recode-INFO-all
	--maf 0.01
	--max-alleles 2
	--min-alleles 2
	--thin 5
	--max-missing 0.5
	--out balsam_miss50_thin5_MAF1
	--recode
	--remove-filtered-all
	--remove-indels

Using zlib version: 1.2.11
After filtering, kept 373 out of 373 Individuals
Outputting VCF file...
After filtering, kept 152319 out of a possible 984452 Sites
Run Time = 290.00 seconds


In [89]:
!$vcftools --remove-indels \
--max-missing 0.7 \
--min-alleles 2 \
--max-alleles 2 \
--maf 0.01 \
--thin 5 \
--remove-filtered-all \
--recode \
--recode-INFO-all \
--gzvcf \
$vcf_file \
--out $'balsam_miss70_thin5_MAF1'


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/TotalRawSNPs.vcf.gz
	--recode-INFO-all
	--maf 0.01
	--max-alleles 2
	--min-alleles 2
	--thin 5
	--max-missing 0.7
	--out balsam_miss70_thin5_MAF1
	--recode
	--remove-filtered-all
	--remove-indels

Using zlib version: 1.2.11
After filtering, kept 373 out of 373 Individuals
Outputting VCF file...
After filtering, kept 115242 out of a possible 984452 Sites
Run Time = 246.00 seconds


In [90]:
!$vcftools --remove-indels \
--max-missing 0.7 \
--min-alleles 2 \
--max-alleles 2 \
--maf 0.01 \
--thin 100 \
--remove-filtered-all \
--recode \
--recode-INFO-all \
--gzvcf \
$vcf_file \
--out $'balsam_miss70_thin100_MAF1'


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/TotalRawSNPs.vcf.gz
	--recode-INFO-all
	--maf 0.01
	--max-alleles 2
	--min-alleles 2
	--thin 100
	--max-missing 0.7
	--out balsam_miss70_thin100_MAF1
	--recode
	--remove-filtered-all
	--remove-indels

Using zlib version: 1.2.11
After filtering, kept 373 out of 373 Individuals
Outputting VCF file...
After filtering, kept 35716 out of a possible 984452 Sites
Run Time = 145.00 seconds


In [91]:
vcf_filtered = "%s/%s.recode.vcf" % (analysis_dir,'balsam_miss70_thin100_MAF1')
vcf_filtered_gz = "%s.gz" % vcf_filtered

In [92]:
!$bgzip -c $vcf_filtered > {vcf_filtered_gz}
!$tabix {vcf_filtered_gz}

# Remove bad_indv


In [96]:
!$vcftools --gzvcf $vcf_filtered_gz --out $vcf_filtered_gz --missing-indv


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/balsam_miss70_thin100_MAF1.recode.vcf.gz
	--missing-indv
	--out /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/balsam_miss70_thin100_MAF1.recode.vcf.gz

Using zlib version: 1.2.11
After filtering, kept 373 out of 373 Individuals
Outputting Individual Missingness
After filtering, kept 35716 out of a possible 35716 Sites
Run Time = 3.00 seconds


In [97]:
def get_imiss(filedir, prefix):
    imiss_file = !ls {filedir}/{prefix}.imiss
    imiss_df = pd.read_csv(imiss_file[0], sep="\t")
    imiss_df.index = imiss_df.INDV
    return imiss_df

In [98]:
imiss_df = get_imiss(analysis_dir,'balsam_miss70_thin100_MAF1.recode.vcf.gz')
imiss_df.head()

,INDV,N_DATA,N_GENOTYPES_FILTERED,N_MISS,F_MISS
INDV,,,,,
BS_AN_10,BS_AN_10,35716,0,3385,0.094775
BS_AN_11,BS_AN_11,35716,0,2514,0.070389
BS_AN_12,BS_AN_12,35716,0,3414,0.095587
BS_AN_1,BS_AN_1,35716,0,2882,0.080692
BS_AN_2,BS_AN_2,35716,0,2273,0.063641


In [99]:
imiss_df.F_MISS.describe()

count    373.000000
mean       0.125503
std        0.117117
min        0.029511
25%        0.082148
50%        0.101635
75%        0.131902
max        0.997116
Name: F_MISS, dtype: float64

In [102]:
len(imiss_df),len(imiss_df[imiss_df.F_MISS >= .4]),len(imiss_df[imiss_df.F_MISS >= .3]), len(imiss_df[imiss_df.F_MISS >= .25])


(373, 6, 11, 16)

In [103]:
bad_indv = imiss_df.INDV[imiss_df.F_MISS >= .3]

In [104]:
with open(os.path.join(analysis_dir, "bad_indv.txt"), "w") as o:
    o.write("INDV\n")
    for elem in bad_indv.index:
        o.write("%s\n" % elem)

In [105]:
!$vcftools --gzvcf $vcf_filtered_gz \
--remove-indels  \
--remove-filtered-all \
--recode \
--recode-INFO-all \
--remove {os.path.join(analysis_dir, "bad_indv.txt")} \
--out {os.path.join(analysis_dir, "snps_indv_removed")}


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/balsam_miss70_thin100_MAF1.recode.vcf.gz
	--remove /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/bad_indv.txt
	--recode-INFO-all
	--out /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/snps_indv_removed
	--recode
	--remove-filtered-all
	--remove-indels

Using zlib version: 1.2.11
Excluding individuals in 'exclude' list
After filtering, kept 362 out of 373 Individuals
Outputting VCF file...
After filtering, kept 35716 out of a possible 35716 Sites
Run Time = 47.00 seconds


In [106]:
vcf_bad_remove = os.path.join(analysis_dir, "snps_indv_removed.recode.vcf")
vcf_bad_remove_gz = vcf_bad_remove + ".gz"
!$bgzip -c {vcf_bad_remove} > {vcf_bad_remove_gz}
!$tabix {vcf_bad_remove_gz}

# Filter snps further 
this needs to be done after removing individuals


In [107]:
def get_vcf_stats(vcf_gz):
    
    stats = ['depth',
            'site-depth',
            'site-mean-depth',
            'site-quality',
            'missing-site',
            'freq',
            'counts',
            'hardy',
            'het']
    
    for stat in stats:
        !$vcftools --gzvcf $vcf_gz \
        --out $vcf_gz \
        {"--%s" % stat} 

In [108]:
get_vcf_stats(vcf_bad_remove_gz)


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/snps_indv_removed.recode.vcf.gz
	--depth
	--out /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/snps_indv_removed.recode.vcf.gz

Using zlib version: 1.2.11
After filtering, kept 362 out of 362 Individuals
Outputting Mean Depth by Individual
After filtering, kept 35716 out of a possible 35716 Sites
Run Time = 3.00 seconds

VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/snps_indv_removed.recode.vcf.gz
	--out /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/snps_indv_removed.recode.vcf.gz
	--site-depth

Using zlib version: 1.2.11
After filtering, kept 362 out of 362 Individuals
Outputting Depth for Each Site


After filtering, kept 35716 out of a possible 35716 Sites
Run Time = 4.00 seconds

VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/snps_indv_removed.recode.vcf.gz
	--out /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/snps_indv_removed.recode.vcf.gz
	--site-mean-depth

Using zlib version: 1.2.11
After filtering, kept 362 out of 362 Individuals
Outputting Depth for Each Site
After filtering, kept 35716 out of a possible 35716 Sites
Run Time = 4.00 seconds

VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/snps_indv_removed.recode.vcf.gz
	--out /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/snps_indv_removed.recode.vcf.gz
	--site-quality

Using zlib version: 1.2.11
After filtering, kept 362 out of 362 Individuals
Outputting Quality for Each Site


After filtering, kept 35716 out of a possible 35716 Sites
Run Time = 3.00 seconds

VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/snps_indv_removed.recode.vcf.gz
	--out /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/snps_indv_removed.recode.vcf.gz
	--missing-site

Using zlib version: 1.2.11
After filtering, kept 362 out of 362 Individuals
Outputting Site Missingness
After filtering, kept 35716 out of a possible 35716 Sites
Run Time = 4.00 seconds

VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/snps_indv_removed.recode.vcf.gz
	--freq
	--out /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/snps_indv_removed.recode.vcf.gz

Using zlib version: 1.2.11
After filtering, kept 362 out of 362 Individuals
Outputting Frequency Statistics...


After filtering, kept 35716 out of a possible 35716 Sites
Run Time = 4.00 seconds

VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/snps_indv_removed.recode.vcf.gz
	--counts
	--out /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/snps_indv_removed.recode.vcf.gz

Using zlib version: 1.2.11
After filtering, kept 362 out of 362 Individuals
Outputting Frequency Statistics...
After filtering, kept 35716 out of a possible 35716 Sites
Run Time = 3.00 seconds

VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/snps_indv_removed.recode.vcf.gz
	--hardy
	--out /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/snps_indv_removed.recode.vcf.gz

Using zlib version: 1.2.11
After filtering, kept 362 out of 362 Individuals
Outputting HWE statistics (but only for biallelic loci)


After filtering, kept 35716 out of a possible 35716 Sites
Run Time = 4.00 seconds

VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/snps_indv_removed.recode.vcf.gz
	--het
	--out /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/snps_indv_removed.recode.vcf.gz

Using zlib version: 1.2.11
After filtering, kept 362 out of 362 Individuals
Outputting Individual Heterozygosity
After filtering, kept 35716 out of a possible 35716 Sites
Run Time = 4.00 seconds


In [109]:
#pd.set_option('display.max_columns', 100)

def get_MAF(row):
    try:
        return np.min([row.A1_freq, row.A2_freq])
    except:
        print(row)
        
def get_correction(n):
    #for finite sample size
    return (2*n)/(2*n-1)

def calculate_Fis(vals):
    try:
        data = [float(x) for x in vals.split("/")]
        assert len(data) == 3
        num_individuals = np.sum(data)
        total_alleles = 2*num_individuals
        a1_count = 2*data[0]
        a2_count = 2*data[2]
        het_count = data[1]
        a1_count += het_count
        a2_count += het_count
        a1_freq = a1_count/total_alleles
        a2_freq = a2_count/total_alleles
        assert a1_freq + a2_freq == 1.0
        He = 2 * a1_freq * a2_freq * get_correction(num_individuals)
        Ho = het_count/num_individuals
        Fis = 1 - (Ho/He)
        return Fis
    except:
        return -9

def combine_vcf_stats(filedir, prefix):
    
    hardy_files = !ls {filedir}/{prefix}.hwe
    hardy = pd.read_csv(hardy_files[0], sep="\t")

    hardy.columns = ['CHROM', 'POS', 'OBS(HOM1/HET/HOM2)', 'E(HOM1/HET/HOM2)', 'ChiSq_HWE',
       'P_HWE', 'P_HET_DEFICIT', 'P_HET_EXCESS']
    hardy.index = hardy.apply(lambda x: "%s-%d" % (x.CHROM, x.POS), axis=1)
    
    loci_files = !ls {filedir}/{prefix}.l* | grep -v log
    loci_df = pd.concat([pd.read_csv(x, sep="\t", skiprows=0) for x in loci_files], axis=1)
    chrom_pos = loci_df.ix[:,0:2]
    
    frq_files = !ls {filedir}/{prefix}.frq* | grep -v count
    frq_data = []
    h = open(frq_files[0])
    header = h.readline().strip().split()
    for line in h:
        frq_data.append(line.strip().split('\t'))

    header = ['CHROM', 'POS', 'N_ALLELES', 'N_CHR', 'A1_FREQ', "A2_FREQ"]
    frq_df = pd.DataFrame(frq_data)
    print(frq_df.columns)
    #frq_df = frq_df.drop([6,7],axis=1)
    frq_df.columns = header
    frq_df.index = frq_df.apply(lambda x: "%s-%s" % (x.CHROM, x.POS), axis=1)
    
    loci_df = loci_df.drop(['CHROM','CHR','POS'], axis=1)
    loci_df = pd.concat([chrom_pos, loci_df], axis=1)
    loci_df.index = loci_df.apply(lambda x: "%s-%d" % (x.CHROM, x.POS), axis=1)
    
    loci_df = pd.concat([loci_df, frq_df, hardy], axis=1)
    loci_df["A1_allele"] = loci_df.apply(lambda row: row.A1_FREQ.split(":")[0], axis=1)
    loci_df["A2_allele"] = loci_df.apply(lambda row: row.A2_FREQ.split(":")[0], axis=1)
    
    loci_df["A1_freq"] = loci_df.apply(lambda row: float(row.A1_FREQ.split(":")[1]), axis=1)
    loci_df["A2_freq"] = loci_df.apply(lambda row: float(row.A2_FREQ.split(":")[1]), axis=1)
    
    loci_df['MAF'] = loci_df.apply(get_MAF, axis=1)
    loci_df = loci_df.drop(['CHROM', 'POS'], axis=1)
    
    loci_df['Fis'] = loci_df['OBS(HOM1/HET/HOM2)'].apply(calculate_Fis)
    
    return loci_df, frq_df, hardy

In [110]:
vcf_bad_remove_gz

'/data/gpfs/home/tfaske/g/balsam/ddocent/filtering/snps_indv_removed.recode.vcf.gz'

In [111]:
loci_df, frq_df, hardy = combine_vcf_stats(analysis_dir,'snps_indv_removed.recode.vcf.gz')

/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:45: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)


RangeIndex(start=0, stop=6, step=1)


In [112]:
loci_df

,SUM_DEPTH,SUMSQ_DEPTH,MEAN_DEPTH,VAR_DEPTH,N_DATA,N_GENOTYPE_FILTERED,N_MISS,F_MISS,QUAL,N_ALLELES,...,ChiSq_HWE,P_HWE,P_HET_DEFICIT,P_HET_EXCESS,A1_allele,A2_allele,A1_freq,A2_freq,MAF,Fis
dDocent_Contig_1545-3,24195,3255087,66.83700,4537.30000,724,0,0,0.000000,0.312898,2,...,0.057356,1.000000e+00,1.000000e+00,0.950929,C,T,0.987569,0.012431,0.012431,-0.011189
dDocent_Contig_5506-57,5079,150467,14.47010,219.92400,724,0,22,0.030387,0.004784,2,...,8.128072,9.116096e-02,9.116096e-02,0.997050,C,A,0.982906,0.017094,0.017094,0.153382
dDocent_Contig_9654-4,5195,144471,14.43060,193.60500,724,0,4,0.005525,24.716800,2,...,1.395282,2.816229e-01,2.816229e-01,0.963134,A,C,0.969444,0.030556,0.030556,0.063558
dDocent_Contig_10078-69,10575,416777,29.29360,297.21400,724,0,2,0.002762,1332.210000,2,...,114.222700,1.421497e-20,1.421497e-20,1.000000,T,G,0.842105,0.157895,0.157895,0.563106
dDocent_Contig_11248-3,10837,606471,29.93650,781.30100,724,0,0,0.000000,38.887800,2,...,0.292162,1.000000e+00,1.000000e+00,0.763586,G,A,0.972376,0.027624,0.027624,-0.026989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
dDocent_Contig_722075-2,1304,14744,4.46575,30.65520,724,0,140,0.193370,94.220200,2,...,93.619750,3.089170e-04,3.089170e-04,1.000000,A,C,0.988014,0.011986,0.011986,0.566972
dDocent_Contig_722076-3,1639,12479,4.90719,13.32170,724,0,56,0.077348,24533.800000,2,...,0.116850,8.224778e-01,6.649764e-01,0.420150,A,G,0.422156,0.577844,0.422156,-0.017179
dDocent_Contig_722104-74,1737,12489,4.94872,11.12310,724,0,22,0.030387,0.263162,2,...,10.590390,3.159676e-03,3.159676e-03,0.999259,T,G,0.891738,0.108262,0.108262,0.174878
dDocent_Contig_722112-16,827,3707,2.91197,4.58940,724,0,156,0.215470,86.666800,2,...,43.029830,1.931778e-03,1.931778e-03,0.999983,T,C,0.982394,0.017606,0.017606,0.390323


In [113]:
frq_df

,CHROM,POS,N_ALLELES,N_CHR,A1_FREQ,A2_FREQ
dDocent_Contig_1545-3,dDocent_Contig_1545,3,2,724,C:0.987569,T:0.0124309
dDocent_Contig_5506-57,dDocent_Contig_5506,57,2,702,C:0.982906,A:0.017094
dDocent_Contig_9654-4,dDocent_Contig_9654,4,2,720,A:0.969444,C:0.0305556
dDocent_Contig_10078-69,dDocent_Contig_10078,69,2,722,T:0.842105,G:0.157895
dDocent_Contig_11248-3,dDocent_Contig_11248,3,2,724,G:0.972376,A:0.0276243
...,...,...,...,...,...,...
dDocent_Contig_722075-2,dDocent_Contig_722075,2,2,584,A:0.988014,C:0.0119863
dDocent_Contig_722076-3,dDocent_Contig_722076,3,2,668,A:0.422156,G:0.577844
dDocent_Contig_722104-74,dDocent_Contig_722104,74,2,702,T:0.891738,G:0.108262
dDocent_Contig_722112-16,dDocent_Contig_722112,16,2,568,T:0.982394,C:0.0176056


In [114]:
hardy

,CHROM,POS,OBS(HOM1/HET/HOM2),E(HOM1/HET/HOM2),ChiSq_HWE,P_HWE,P_HET_DEFICIT,P_HET_EXCESS
dDocent_Contig_1545-3,dDocent_Contig_1545,3,353/9/0,353.06/8.89/0.06,0.057356,1.000000e+00,1.000000e+00,0.950929
dDocent_Contig_5506-57,dDocent_Contig_5506,57,340/10/1,339.10/11.79/0.10,8.128072,9.116096e-02,9.116096e-02,0.997050
dDocent_Contig_9654-4,dDocent_Contig_9654,4,339/20/1,338.34/21.33/0.34,1.395282,2.816229e-01,2.816229e-01,0.963134
dDocent_Contig_10078-69,dDocent_Contig_10078,69,283/42/36,256.00/96.00/9.00,114.222700,1.421497e-20,1.421497e-20,1.000000
dDocent_Contig_11248-3,dDocent_Contig_11248,3,342/20/0,342.28/19.45/0.28,0.292162,1.000000e+00,1.000000e+00,0.763586
...,...,...,...,...,...,...,...,...
dDocent_Contig_722075-2,dDocent_Contig_722075,2,287/3/2,285.04/6.92/0.04,93.619750,3.089170e-04,3.089170e-04,1.000000
dDocent_Contig_722076-3,dDocent_Contig_722076,3,58/166/110,59.52/162.95/111.52,0.116850,8.224778e-01,6.649764e-01,0.420150
dDocent_Contig_722104-74,dDocent_Contig_722104,74,285/56/10,279.11/67.77/4.11,10.590390,3.159676e-03,3.159676e-03,0.999259
dDocent_Contig_722112-16,dDocent_Contig_722112,16,276/6/2,274.09/9.82/0.09,43.029830,1.931778e-03,1.931778e-03,0.999983


In [115]:
loci_out = os.path.join(analysis_dir,'loci_out.csv')
frq_out = os.path.join(analysis_dir,'frq_out.csv')
hardy_out = os.path.join(analysis_dir,'hardy_out.csv')

loci_df.to_csv(path_or_buf=loci_out)
frq_df.to_csv(path_or_buf=frq_out)
hardy.to_csv(path_or_buf=hardy_out)


In [116]:
loci_df.SUM_DEPTH.describe()

count     35716.000000
mean       4396.504620
std       18284.307609
min         560.000000
25%        1209.000000
50%        1922.000000
75%        3338.000000
max      555103.000000
Name: SUM_DEPTH, dtype: float64

In [117]:
loci_df.QUAL.describe()

count    3.571600e+04
mean     5.590921e+03
std      1.087277e+05
min      0.000000e+00
25%      1.935793e+02
50%      6.189120e+02
75%      2.087733e+03
max      9.046740e+06
Name: QUAL, dtype: float64

In [118]:
len(loci_df[loci_df.Fis == -9])

0

In [119]:
len(loci_df[loci_df.SUM_DEPTH <5000]),len(loci_df[loci_df.SUM_DEPTH >=1000])

(31124, 30234)

In [120]:
len(loci_df[loci_df.QUAL >= 10]) - len(loci_df[loci_df.QUAL >= 20])

361

In [121]:
len(loci_df[loci_df.QUAL <  500]), len(loci_df[loci_df.QUAL < 200]), len(loci_df[loci_df.QUAL < 100])

(16222, 9134, 5009)

In [124]:
 len(loci_df[loci_df.Fis <= -0.5]), len(loci_df[loci_df.MAF < 0.01])

(222, 230)

In [127]:
def filter_snps(df, imputed=False):
    if imputed:
        return df[(df.MAF >= 0.01) &  
                  (df.Fis > -0.5)]
    else:
        return df[(df.SUM_DEPTH >= 1000) & 
                  (df.SUM_DEPTH < 5000) & 
                  (df.QUAL >= 200) & 
                  (df.MAF >= 0.01) &  
                  (df.Fis > -0.5)]

In [128]:
loci_stage1 = filter_snps(loci_df)
loci_stage1.shape

(19200, 25)

In [129]:
with open(os.path.join(analysis_dir, "stage1_positions.txt"), "w") as o:
    for elem in loci_stage1.index:
        o.write("%s\n" % "\t".join(elem.split("-")))

In [130]:
!$vcftools --gzvcf $vcf_bad_remove_gz \
--remove-indels  \
--remove-filtered-all \
--recode \
--recode-INFO-all \
--positions {os.path.join(analysis_dir, "stage1_positions.txt")} \
--out {os.path.join(analysis_dir, "good_snps")}


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/snps_indv_removed.recode.vcf.gz
	--recode-INFO-all
	--out /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/good_snps
	--positions /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/stage1_positions.txt
	--recode
	--remove-filtered-all
	--remove-indels

Using zlib version: 1.2.11
After filtering, kept 362 out of 362 Individuals
Outputting VCF file...
After filtering, kept 19200 out of a possible 35716 Sites
Run Time = 27.00 seconds


In [131]:
snps = os.path.join(analysis_dir, "good_snps.recode.vcf")
snps_gz = snps + ".gz"
!$bgzip -c {snps} > {snps_gz}
!$tabix {snps_gz}

# Plink / chrom_map

In [132]:
chroms = sorted(set([x.split("-")[0] for x in loci_df.index]))
len(chroms) # number of unique contigs

35716

In [133]:
with open(os.path.join(analysis_dir, "chrom_map.txt"), "w") as o:
    for i, c in enumerate(chroms):
        o.write("%s\t%d\n" % (c, i))

In [134]:
def write_plink_files(vcf_gz):
    !$vcftools --gzvcf {vcf_gz} \
    --out {vcf_gz} \
    --plink \
    --chrom-map {os.path.join(analysis_dir, "chrom_map.txt")}

In [135]:
write_plink_files(snps_gz)


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/good_snps.recode.vcf.gz
	--chrom-map /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/chrom_map.txt
	--out /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/good_snps.recode.vcf.gz
	--plink

Using zlib version: 1.2.11
After filtering, kept 362 out of 362 Individuals
Writing PLINK PED and MAP files ... 
	Read 35717 chromosome mapping file entries.
Done.
After filtering, kept 19200 out of a possible 19200 Sites
Run Time = 4.00 seconds


# Make 012

In [136]:
f = os.path.join(analysis_dir, "good_snps.recode.vcf")
!$vcftools --gzvcf {f} \
--out {f} \
--012


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/good_snps.recode.vcf
	--012
	--out /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/good_snps.recode.vcf

Using zlib version: 1.2.11
After filtering, kept 362 out of 362 Individuals
Writing 012 matrix files ... Done.
After filtering, kept 19200 out of a possible 19200 Sites
Run Time = 2.00 seconds


# Get coverage per individual

In [137]:
f = os.path.join(analysis_dir, "good_snps.recode.vcf")
!$vcftools --gzvcf {f} \
--out {f} \
--depth


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/good_snps.recode.vcf
	--depth
	--out /data/gpfs/home/tfaske/g/balsam/ddocent/filtering/good_snps.recode.vcf

Using zlib version: 1.2.11
After filtering, kept 362 out of 362 Individuals
Outputting Mean Depth by Individual
After filtering, kept 19200 out of a possible 19200 Sites
Run Time = 2.00 seconds


In [138]:
depth_file = os.path.join(analysis_dir, "good_snps.recode.vcf.idepth")
depth_df = pd.read_csv(depth_file, sep="\t")
depth_df.head()

,INDV,N_SITES,MEAN_DEPTH
0,BS_AN_10,17462,7.09529
1,BS_AN_11,17942,6.40770
2,BS_AN_12,17549,6.67833
3,BS_AN_1,17604,7.29743
4,BS_AN_2,17927,9.36955


In [139]:
depth_df.MEAN_DEPTH.describe()

count    362.000000
mean       7.041527
std        1.233835
min        3.023500
25%        6.329185
50%        7.070525
75%        7.810755
max       12.882800
Name: MEAN_DEPTH, dtype: float64

# Remove unnecessary files

In [140]:
!rm snps*

In [141]:
!rm balsam*

In [142]:
!rm *vcf